In [6]:
from PIL import Image, ImageDraw, ImageFont, ImageOps, ExifTags
import os
import piexif
from datetime import datetime

def correct_orientation(image):
    """
    Correct the orientation of an image using its EXIF data.
    """
    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == 'Orientation':
            break

    exif = image._getexif()

    if exif is not None and orientation in exif:
        if exif[orientation] == 3:
            image = image.rotate(180, expand=True)
        elif exif[orientation] == 6:
            image = image.rotate(270, expand=True)
        elif exif[orientation] == 8:
            image = image.rotate(90, expand=True)

    return image

def add_watermark(image_path, text, save_as=None):
    original = Image.open(image_path)
    original = correct_orientation(original)
    txt = Image.new('RGBA', original.size, (255,255,255,0))
    draw = ImageDraw.Draw(txt)

    # Scale the font size based on the image size
    base_width = 800  # Change this to your preferred base width
    font_size = int((min(original.size) / base_width) * 20)
    font_size = max(font_size, 10)  # Set a minimum font size if needed

    font = ImageFont.truetype("arial.ttf", font_size)
    text_x = original.width // 2
    text_y = original.height - 30

    text_position = (text_x, text_y)
    draw.text(text_position, text, fill=(255,255,255,128), font=font, anchor="mm")
    watermarked = Image.alpha_composite(original.convert('RGBA'), txt).convert('RGB')
    
    if save_as:
        watermarked.save(save_as, 'JPEG')
    else:
        watermarked.save(image_path, 'JPEG')

def get_camera_info(exif_data):
    camera_info = {}
    # Only add EXIF data to the dictionary if it is not the default 'Unknown'
    if piexif.ImageIFD.Model in exif_data['0th']:
        camera_info['model'] = exif_data['0th'][piexif.ImageIFD.Model].decode('utf-8')

    if piexif.ExifIFD.ISOSpeedRatings in exif_data['Exif']:
        camera_info['iso'] = str(exif_data['Exif'][piexif.ExifIFD.ISOSpeedRatings])

    if piexif.ExifIFD.FNumber in exif_data['Exif']:
        f_number = exif_data['Exif'][piexif.ExifIFD.FNumber]
        camera_info['fstop'] = f"{f_number[0] / f_number[1]}"

    if piexif.ExifIFD.ExposureTime in exif_data['Exif']:
        exposure_time = exif_data['Exif'][piexif.ExifIFD.ExposureTime]
        camera_info['shutterspeed'] = f"{exposure_time[0]}/{exposure_time[1]} sec"

    if piexif.ExifIFD.DateTimeOriginal in exif_data['Exif']:
        try:
            camera_info['datetime'] = exif_data['Exif'][piexif.ExifIFD.DateTimeOriginal].decode('utf-8')
            camera_info['year'] = datetime.strptime(camera_info['datetime'], '%Y:%m:%d %H:%M:%S').year
        except:
            pass

    return camera_info

def process_images(source_folder_path):
    target_folder_path = '../images/watermark'
    os.makedirs(target_folder_path, exist_ok=True)

    for filename in os.listdir(source_folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(source_folder_path, filename)
            exif_data = piexif.load(image_path)
            camera_info = get_camera_info(exif_data)

            # Construct watermark text
            watermark_elements = []
            for key, value in camera_info.items():
                if key == 'model':  # Only include the value for 'model'
                    watermark_elements.append(f"{value}")
                elif key == 'iso':  # Include 'ISO' and its value
                    watermark_elements.append(f"ISO {value}")
                elif key == 'shutterspeed':  # Only include the value for 'shutterspeed'
                    watermark_elements.append(f"{value}")
            watermark_text = f"(c) Charles Lehnen, " + ', '.join(watermark_elements)

            save_as_path = os.path.join(target_folder_path, f"{os.path.splitext(filename)[0]}_watermarked.jpg")
            add_watermark(image_path, watermark_text, save_as=save_as_path)

process_images('../images/no_watermark')


In [7]:
import os

# Directory where the images are stored
image_folder = '../images/watermark/'
# Prefix for the path to use in the markdown output
output_path_prefix = 'images/watermark/'

images = [f for f in os.listdir(image_folder) if f.lower().endswith('.jpg')]

markdown_output = "::: {layout=\"[[1, 1], [1, 1]]\"}\n"
for image in images:
    # Correct the path for the output
    image_path = os.path.join(output_path_prefix, image).replace("\\", "/")
    # Add an image entry with an empty line after it
    markdown_output += f"![]({image_path}){{group=\"my-gallery\"}}\n\n"
markdown_output = markdown_output.strip() + "\n:::"

print(markdown_output)



::: {layout="[[1, 1], [1]]"}
![](images/watermark/amblyrhynchus_cristatus_watermarked.jpg){group="my-gallery"}

![](images/watermark/california_sea_lion.JPG_watermarked_watermarked.jpg){group="my-gallery"}

![](images/watermark/california_sea_lion_watermarked.jpg){group="my-gallery"}

![](images/watermark/conolophus_subcristalus_watermarked.jpg){group="my-gallery"}

![](images/watermark/iguana_iguana_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_0110_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_3895_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5437_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5444_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5670_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5868_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5902_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_7778_watermarked.jpg){group="my-gallery"}

![](images/wat